# El corrido de Gregorio Cortez

## Mapping the manhunt of Rio Grande border folk hero Gregorio Cortez:  the largest manhunt in U.S. history.

### June 14, 1901 to June 22, 1901

### based on lyrics from the Corrido de Gregorio Cortez (variant x english translation by Americo Paredes)

#### project author Diane López 

In [6]:
import geopandas as gpd
from pyproj import CRS
import pandas as pd
import numpy as np

from geograpy import places
import re

import geopy
from geopy.geocoders import ArcGIS
#from geopy.extra.rate_limiter import RateLimiter

import shapely
from shapely.geometry import Point
from shapely.wkt import loads
import shapely.wkt
from shapely import wkt

import contextily as cx
import xyzservices.providers as xyz
import matplotlib.pyplot as plt #to make sure there are no errors when plotting a graph
import pyproj

#import nltk

import spacy
from spacy import displacy

import googlemaps

import locationtagger
nlp = spacy.load("en_core_web_sm")

In [7]:
text = "corrido corpus\ElCorridodeGregorioCortez_X.txt"
with open(text, 'r', encoding='utf-8') as c:
    text = c.read()
    
def clean_text(text):
    cleaned= re.sub(r'[":;,.“”]', "", text)
    return(cleaned)
text = clean_text(text)
#print(text)


TxGPE=[]
nlp = spacy.load("tx_trained_ner")
doc =nlp(text) 
#print(doc)
for ent in doc.ents:
    #print(ent.text, ent.label_)
    if ent.label_ == "GPE":
        TxGPE.append(ent.text)
print(TxGPE)

['Karnes', 'Gonzales', 'Belmont', 'Laredo', 'Encinal']


In [8]:
ents = [(e.text, e.start_char, e.end_char, e.label_)for e in doc.ents]
print(ents)

ents = [(e.text)for e in doc.ents]
print(ents)

[('Karnes', 18, 24, 'GPE'), ('Gonzales', 1051, 1059, 'GPE'), ('Belmont', 1140, 1147, 'GPE'), ('Laredo', 1550, 1556, 'GPE'), ('Encinal', 1864, 1871, 'GPE')]
['Karnes', 'Gonzales', 'Belmont', 'Laredo', 'Encinal']


In [ ]:
displacy.render(doc, style ='ent', jupyter=True, page=True)

Geoparsing: Finding places from the corrido lyrics

In [ ]:
pc = places.PlaceContext(ents)

pc.set_countries()
print (pc.countries) #['United States']

pc.set_regions()
print(pc.regions) #['Texas'])

pc.set_cities()
print(pc.cities) #['Brownsville'])

#print(pc.address_strings) #['Brownsville','Texas, United States'])
#print(pc.cities[0:5])

Pandas Dataframe
NER Places Geolocation 

In [43]:
df = pd.DataFrame(ents, columns=["NER_Places"])
geolocator = ArcGIS(user_agent='CorridosMap')
geocode = lambda query: geolocator.geocode("%s, Texas" % query)
df['Location'] = df['NER_Places'].apply(geocode)
#df['geometry'] = df['NER_Places'].apply(geocode)
df

,NER_Places,Location
0,Karnes,"(Karnes County, Texas, (28.905630820000056, -9..."
1,Gonzales,"(Gonzales, Texas, (29.500150000000076, -97.452..."
2,Belmont,"(Belmont, Texas, (29.52287000000007, -97.68328..."
3,Laredo,"(Laredo, Texas, (27.530920000000037, -99.50230..."
4,Encinal,"(Encinal, Texas, (28.046230000000037, -99.3566..."


In [54]:
gdf = gpd.tools.geocode(df.Location, provider='ArcGIS')
gdf


,geometry,address
0,POINT (-97.85942 28.90563),"Karnes County, Texas"
1,POINT (-97.45223 29.50015),"Gonzales, Texas"
2,POINT (-97.68329 29.52287),"Belmont, Texas"
3,POINT (-99.50231 27.53092),"Laredo, Texas"
4,POINT (-99.35670 28.04623),"Encinal, Texas"


In [ ]:

px.set_mapbox_access_token(open("mapboxtoken").read())
fig = px.line_mapbox(gdf,
                        lat=gdf.geometry.y,
                        lon=gdf.geometry.x,
                        hover_name="address",
                        )

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=6, mapbox_center_lat = 29,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [31]:
import plotly.graph_objects as go

token = open("mapboxtoken").read() # you need your own token

fig = go.Figure(go.Scattermapbox(
    data_frame= gdf,
    mode = "markers+text+lines",
    lon =gdf.geometry.x , lat = gdf.geometry.y,
    marker = {'size': 20, 'symbol': ["bus", "harbor", "airport"]},
    text = ["Bus", "Harbor", "airport"],textposition = "bottom right"))

fig.update_layout(
    mapbox = {
        'accesstoken': token,
        'style': "outdoors", 'zoom': 0.7},
    showlegend = False)

fig.show()

ValueError: Invalid property specified for object of type plotly.graph_objs.Scattermapbox: 'data'

Did you mean "lat"?

    Valid properties:
        below
            Determines if this scattermapbox trace's layers are to
            be inserted before the layer with the specified ID. By
            default, scattermapbox layers are inserted above all
            the base layers. To place the scattermapbox layers
            above every other layer, set `below` to "''".
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on Chart Studio Cloud for
            `customdata`.
       